In [1]:
MAX_LEN = 26

In [2]:
import os
import pprint
import numpy as np
import pandas as pd
import deepdish as dd

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader

import optuna
from optuna.integration import PyTorchLightningPruningCallback
from torchnlp.encoders.text import CharacterEncoder, StaticTokenizerEncoder

In [3]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, roc_auc_score, roc_curve, matthews_corrcoef, plot_confusion_matrix
from collections import defaultdict

In [ ]:
def encode(seq, max_len):
    nucl_dict = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    mat = np.zeros((4, max_len), dtype=int)
    
    for i, nucl in enumerate(seq):
        mat[nucl_dict[nucl]][i] = 1

    return mat

def encode_pair(seq1, seq2, max_len):
    enc1 = encode(seq1, max_len)
    enc2 = encode(seq2, max_len)
    return np.array([enc1, enc2])

In [6]:
# Adapted froms PyTorch forums
import math

def num2tuple(num):
    return num if isinstance(num, tuple) else (num, num)

def conv2d_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    h_w, kernel_size, stride, pad, dilation = num2tuple(h_w), \
        num2tuple(kernel_size), num2tuple(stride), num2tuple(pad), num2tuple(dilation)
    pad = num2tuple(pad[0]), num2tuple(pad[1])
    
    h = math.floor((h_w[0] + sum(pad[0]) - dilation[0]*(kernel_size[0]-1) - 1) / stride[0] + 1)
    w = math.floor((h_w[1] + sum(pad[1]) - dilation[1]*(kernel_size[1]-1) - 1) / stride[1] + 1)
    
    return h, w

In [7]:
class CNNRegression(pl.LightningModule):
    def __init__(self, trial):
        super(CNNRegression, self).__init__()
        self.out_predictions = []
        self.trial = trial
        
        self.param_trial_dict = {}
        self.out_size = 0

        self.param_trial_dict['conv2d_size'] = trial.suggest_int('conv2d_size', 9, 13)
        self.param_trial_dict['conv2d_filter'] = trial.suggest_int('conv2d_filter', 256, 768)
        self.param_trial_dict['conv2d_dropout'] = trial.suggest_float('conv2d_dropout', 0.1, 0.5)

        self.param_trial_dict['conv1d_size1'] = trial.suggest_int('conv1d_size1', 7, 11)
        self.param_trial_dict['conv1d_size2'] = trial.suggest_int('conv1d_size2', 3, 7)
        self.param_trial_dict['conv1d_size3'] = trial.suggest_int('conv1d_size3', 3, 7)
        self.param_trial_dict['conv1d_size4'] = trial.suggest_int('conv1d_size4', 1, 3)

        self.param_trial_dict['conv1d_filter1'] = trial.suggest_int('conv1d_filter1', 256, 768)
        self.param_trial_dict['conv1d_filter2'] = trial.suggest_int('conv1d_filter2', 256, 768)
        self.param_trial_dict['conv1d_filter3'] = trial.suggest_int('conv1d_filter3', 128, 384)
        self.param_trial_dict['conv1d_filter4'] = trial.suggest_int('conv1d_filter4', 32, 128)

        self.param_trial_dict['conv1d_dropout1'] = trial.suggest_float('conv1d_dropout1', 0.0, 0.5)
        self.param_trial_dict['conv1d_dropout2'] = trial.suggest_float('conv1d_dropout2', 0.0, 0.5)
        self.param_trial_dict['conv1d_dropout3'] = trial.suggest_float('conv1d_dropout3', 0.0, 0.5)
        self.param_trial_dict['conv1d_dropout4'] = trial.suggest_float('conv1d_dropout4', 0.0, 0.5)

        self.out_size = conv2d_output_shape((4, MAX_LEN), kernel_size=(4, self.param_trial_dict['conv2d_size']))
        self.out_size = conv2d_output_shape(self.out_size, kernel_size=(1, self.param_trial_dict['conv1d_size1']))
        self.out_size = conv2d_output_shape(self.out_size, kernel_size=(1, self.param_trial_dict['conv1d_size2']))
        self.out_size = conv2d_output_shape(self.out_size, kernel_size=(1, self.param_trial_dict['conv1d_size3']))
        self.out_size = conv2d_output_shape(self.out_size, kernel_size=(1, self.param_trial_dict['conv1d_size4']))
        self.out_size = self.out_size[1]
            
        if self.out_size < 3:
            raise optuna.TrialPruned()
            
        print('Trial parameters:')
        pprint.PrettyPrinter(indent=4).pprint(self.param_trial_dict)
                    
        self.conv2d_block = nn.Sequential(
            nn.Conv2d(2, self.param_trial_dict['conv2d_filter'], (4, self.param_trial_dict['conv2d_size'])),
            nn.ReLU(),
            nn.BatchNorm2d(self.param_trial_dict['conv2d_filter']),
            nn.Dropout2d(self.param_trial_dict['conv2d_dropout']),
        )
        
        self.conv1d_block = nn.Sequential(
            nn.Conv1d(self.param_trial_dict['conv2d_filter'], self.param_trial_dict['conv1d_filter1'], self.param_trial_dict['conv1d_size1']),
            nn.ReLU(),
            nn.Dropout(self.param_trial_dict['conv1d_dropout1']),
            nn.BatchNorm1d(self.param_trial_dict['conv1d_filter1']),
            nn.Conv1d(self.param_trial_dict['conv1d_filter1'], self.param_trial_dict['conv1d_filter2'], self.param_trial_dict['conv1d_size2']),
            nn.ReLU(),
            nn.Dropout(self.param_trial_dict['conv1d_dropout2']),
            nn.BatchNorm1d(self.param_trial_dict['conv1d_filter2']),
            nn.Conv1d(self.param_trial_dict['conv1d_filter2'], self.param_trial_dict['conv1d_filter3'], self.param_trial_dict['conv1d_size3']),
            nn.ReLU(),
            nn.Dropout(self.param_trial_dict['conv1d_dropout3']),
            nn.BatchNorm1d(self.param_trial_dict['conv1d_filter3']),
            nn.Conv1d(self.param_trial_dict['conv1d_filter3'], self.param_trial_dict['conv1d_filter4'], self.param_trial_dict['conv1d_size4']),
            nn.ReLU(),
            nn.Dropout(self.param_trial_dict['conv1d_dropout4']),
            nn.BatchNorm1d(self.param_trial_dict['conv1d_filter4']),
        )
        
        print(self.conv1d_block)
                
        self.param_trial_dict['lin_size1'] = trial.suggest_int('lin_size1', max(self.param_trial_dict['conv1d_filter4'] * self.out_size - 500, 0), self.param_trial_dict['conv1d_filter4'] * self.out_size)
        self.param_trial_dict['lin_size2'] = trial.suggest_int('lin_size2', 16, max(self.param_trial_dict['lin_size1'] - 50, 32))
        
        self.param_trial_dict['lin_dropout1'] = trial.suggest_float('lin_dropout1', 0.1, 0.5)
        self.param_trial_dict['lin_dropout2'] = trial.suggest_float('lin_dropout2', 0.1, 0.5)
            
        print('Flattened size: %d' % (self.param_trial_dict['conv1d_filter4'] * self.out_size,))
        self.lin_block = nn.Sequential(
            nn.Linear(self.param_trial_dict['conv1d_filter4'] * self.out_size, self.param_trial_dict['lin_size1']),
            nn.ReLU(),
            nn.Dropout(self.param_trial_dict['lin_dropout1']),
            nn.Linear(self.param_trial_dict['lin_size1'], self.param_trial_dict['lin_size2']),
            nn.ReLU(),
            nn.Dropout(self.param_trial_dict['lin_dropout2']),
            nn.Linear(self.param_trial_dict['lin_size2'], 1)
        )
        
        self.param_trial_dict['lr'] = self.trial.suggest_loguniform('lr', 0.0001, 0.1)
        self.param_trial_dict['batch_size'] = trial.suggest_categorical('batch_size', (256, 512, 1024, 2048, 4096, 8096))
        

    def forward(self, x):
        x = self.conv2d_block(x)
        x = self.conv1d_block(torch.squeeze(x))
        x = x.view(x.size(0), -1)
        x = self.lin_block(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        loss = F.mse_loss(torch.squeeze(self(x)), y)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.param_trial_dict['lr'])
            
    def validation_step(self, batch, batch_idx):
        x, y = batch
        val_loss = F.mse_loss(torch.squeeze(self(x)), y)
        self.log('val_loss', val_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return val_loss
            
    def test_step(self, batch, batch_idx):
        x, y = batch
        test_out = self(x)
        self.out_predictions.append(test_out)
        test_loss = F.mse_loss(torch.squeeze(test_out), y)
        self.log('test_loss', test_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return test_loss
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            list(zip(X_train, y_train)),
            batch_size=self.param_trial_dict['batch_size'],
            shuffle=True,
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            list(zip(X_val, y_val)),
            batch_size=self.param_trial_dict['batch_size'],
            shuffle=False,
        )
    
    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            list(zip(X_test, y_test)),
            batch_size=self.param_trial_dict['batch_size'],
            shuffle=False,
        )

In [8]:
train = dd.io.load('../splits/train.h5')
val = dd.io.load('../splits/val.h5')
test = dd.io.load('../splits/test.h5')
y_train = np.load('../splits/y_train.npy')
y_val = np.load('../splits/y_val.npy')
y_test = np.load('../splits/y_test.npy')

In [9]:
y_train = y_train * 100
y_val = y_val * 100
y_test = y_test * 100

In [10]:
X_train = [encode_pair(item[0], item[1], MAX_LEN) for item in train]
X_val = [encode_pair(item[0], item[1], MAX_LEN) for item in val]
X_test = [encode_pair(item[0], item[1], MAX_LEN) for item in test]

In [11]:
X_train = np.array(X_train, dtype=np.dtype('d'))
X_val = np.array(X_val, dtype=np.dtype('d'))
X_test = np.array(X_test, dtype=np.dtype('d'))

In [12]:
torch.set_default_tensor_type(torch.DoubleTensor)
torch.set_default_dtype(torch.double)

In [13]:
class MetricsCallback(pl.Callback):
    """PyTorch Lightning metric callback."""

    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [14]:
DIR = os.getcwd()
MODEL_DIR = os.path.join(DIR, "model_saved")

In [15]:
def objective(trial):
    # Filenames for each trial must be made unique in order to access each checkpoint.
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        os.path.join(MODEL_DIR, "trial_{}".format(trial.number), "{epoch}"), monitor="val_loss"
    )

    # The default logger in PyTorch Lightning writes to event files to be consumed by
    # TensorBoard. We don't use any logger here as it requires us to implement several abstract
    # methods. Instead we setup a simple callback, that saves metrics from each validation step.
    metrics_callback = MetricsCallback()
    trainer = pl.Trainer(
        max_epochs=5,
        gpus=[0],
        callbacks=[checkpoint_callback, metrics_callback, PyTorchLightningPruningCallback(trial, monitor="val_loss")],
    )

    model = CNNRegression(trial)
    trainer.fit(model)

    return metrics_callback.metrics[-1]["val_loss"].item()

In [16]:
pruner = optuna.pruners.HyperbandPruner()

study = optuna.create_study(direction="minimize", pruner=pruner)
study.optimize(objective, show_progress_bar=True)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-01-28 18:59:38,586] A new study created in memory with name: no-name-cea013f7-15e2-41e6-a5e3-8429cb632977
C:\Users\david\miniconda3\envs\pt1.8-cuda11.1\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


0it [00:00, ?it/s]

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[I 2021-01-28 18:59:38,641] Trial 0 pruned. 
[I 2021-01-28 18:59:38,644] Trial 1 pruned. 
[I 2021-01-28 18:59:38,649] Trial 2 pruned. 
[I 2021-01-28 18:59:38,652] Trial 3 pruned. 
[I 2021-01-28 18:59:38,655] Trial 4 pruned. 
[I 2021-01-28 18:59:38,659] Trial 5 pruned. 
Trial parameters:
{   'conv1d_dropout1': 0.45390805833966574,
    'conv1d_dropout2': 0.47585675153757695,
    'conv1d_dropout3': 0.4083015860453671,
    'conv1d_dropout4': 0.3054092411048428,
    'conv1d_filter1': 659,
    'conv1d_filter2': 475,
    'conv1d_filter3': 348,
    'conv1d_filter4': 116,
    'conv1d_size1': 7,
    'conv1d_size2': 5,
    'conv1d_size3': 3,
    'conv1d_size4': 1,
    'conv2d_dropout': 0.4570931393837364,
    'conv2d_filter': 698,
    'conv2d_size': 9}
Sequential(
  (0): Conv1d(698, 659, kernel_size=(7,), stride=(1,))
  (1): ReLU()
  (2): Dropout(p=0.45390805833966574, inplace=False)
  (3): BatchNorm1d(659, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Conv1d(659, 475, ke


  | Name         | Type       | Params
--------------------------------------------
0 | conv2d_block | Sequential | 52.4 K
1 | conv1d_block | Sequential | 5.3 M 
2 | lin_block    | Sequential | 413 K 
--------------------------------------------
5.8 M     Trainable params
0         Non-trainable params
5.8 M     Total params


...

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | conv2d_block | Sequential | 45.3 K
1 | conv1d_block | Sequential | 6.1 M 
2 | lin_block    | Sequential | 29.0 K
--------------------------------------------
6.2 M     Trainable params
0         Non-trainable params
6.2 M     Total params


[I 2021-01-31 19:47:02,965] Trial 90 finished with value: 0.42995643615722656 and parameters: {'conv2d_size': 10, 'conv2d_filter': 623, 'conv2d_dropout': 0.4168688755057171, 'conv1d_size1': 9, 'conv1d_size2': 4, 'conv1d_size3': 4, 'conv1d_size4': 1, 'conv1d_filter1': 687, 'conv1d_filter2': 726, 'conv1d_filter3': 251, 'conv1d_filter4': 98, 'conv1d_dropout1': 0.03417123500487315, 'conv1d_dropout2': 0.0012576702621896474, 'conv1d_dropout3': 0.44368624586529454, 'conv1d_dropout4': 0.37445924176084855, 'lin_size1': 156, 'lin_size2': 81, 'lin_dropout1': 0.41409200128067714, 'lin_dropout2': 0.17582465988880505, 'lr': 0.0002287922409677974, 'batch_size': 256}. Best is trial 90 with value: 0.42995643615722656.
Trial parameters:
{   'conv1d_dropout1': 0.028492453757041666,
    'conv1d_dropout2': 0.034894955618237106,
    'conv1d_dropout3': 0.4039112231418402,
    'conv1d_dropout4': 0.37549181900621836,
    'conv1d_filter1': 688,
    'conv1d_filter2': 724,
    'conv1d_filter3': 251,
    'conv1d_f

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

KeyboardInterrupt: 

In [18]:
study.best_trial

FrozenTrial(number=90, values=[0.42995643615722656], datetime_start=datetime.datetime(2021, 1, 31, 19, 45, 53, 481214), datetime_complete=datetime.datetime(2021, 1, 31, 19, 47, 2, 964531), params={'conv2d_size': 10, 'conv2d_filter': 623, 'conv2d_dropout': 0.4168688755057171, 'conv1d_size1': 9, 'conv1d_size2': 4, 'conv1d_size3': 4, 'conv1d_size4': 1, 'conv1d_filter1': 687, 'conv1d_filter2': 726, 'conv1d_filter3': 251, 'conv1d_filter4': 98, 'conv1d_dropout1': 0.03417123500487315, 'conv1d_dropout2': 0.0012576702621896474, 'conv1d_dropout3': 0.44368624586529454, 'conv1d_dropout4': 0.37445924176084855, 'lin_size1': 156, 'lin_size2': 81, 'lin_dropout1': 0.41409200128067714, 'lin_dropout2': 0.17582465988880505, 'lr': 0.0002287922409677974, 'batch_size': 256}, distributions={'conv2d_size': IntUniformDistribution(high=13, low=9, step=1), 'conv2d_filter': IntUniformDistribution(high=768, low=256, step=1), 'conv2d_dropout': UniformDistribution(high=0.5, low=0.1), 'conv1d_size1': IntUniformDistrib

In [19]:
study.best_params

{'conv2d_size': 10,
 'conv2d_filter': 623,
 'conv2d_dropout': 0.4168688755057171,
 'conv1d_size1': 9,
 'conv1d_size2': 4,
 'conv1d_size3': 4,
 'conv1d_size4': 1,
 'conv1d_filter1': 687,
 'conv1d_filter2': 726,
 'conv1d_filter3': 251,
 'conv1d_filter4': 98,
 'conv1d_dropout1': 0.03417123500487315,
 'conv1d_dropout2': 0.0012576702621896474,
 'conv1d_dropout3': 0.44368624586529454,
 'conv1d_dropout4': 0.37445924176084855,
 'lin_size1': 156,
 'lin_size2': 81,
 'lin_dropout1': 0.41409200128067714,
 'lin_dropout2': 0.17582465988880505,
 'lr': 0.0002287922409677974,
 'batch_size': 256}

In [20]:
study.best_value

0.42995643615722656